In [1]:
%load_ext autoreload
%autoreload 2
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2


## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    #LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.lfp_electrode import LFPElectrodeGroup
from spyglass.lfp.analysis.v1 import LFPBandSelection,LFPBandV1
from spyglass.lfp.lfp_merge import LFPOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)

[2023-10-26 11:21:11,722][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-10-26 11:21:11,764][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306


In [2]:
LFPV1()#.fetch('nwb_file_name')#&{'nwb_file_name':"Banner20211228_.nwb",}
LFPElectrodeGroup()

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes
arthur20220315_.nwb,all_tets_arthur
arthur20220316_.nwb,all_tets_arthur
arthur20220317_.nwb,all_tets_arthur
arthur20220318_.nwb,all_tets_arthur
arthur20220319_.nwb,all_tets_arthur
arthur20220320_.nwb,all_tets_arthur
arthur20220321_.nwb,all_tets_arthur
arthur20220322_.nwb,all_tets_arthur
arthur20220323_.nwb,all_tets_arthur
arthur20220324_.nwb,all_tets_arthur


## Define Animls


In [6]:
# 'Yoshi20220519_.nwb'
# tetrode_animals = [
#     #Yoshi
#     "Yoshi20220509_.nwb", #permissions
#     "Yoshi20220510_.nwb",
#     "Yoshi20220511_.nwb",
#     "Yoshi20220513_.nwb",
#     "Yoshi20220516_.nwb",
#     "Yoshi20220517_.nwb",
#     "Yoshi20220518_.nwb",
#     # "Yoshi20220519_.nwb", #not inserted
#     # "Yoshi20220525_.nwb", #not inserted
#     #Totoro
#     # "Totoro20220524_.nwb",
#     # "Totoro20220525_.nwb",
#     "Totoro20220526_.nwb",
#     "Totoro20220527_.nwb",
#     "Totoro20220530_.nwb",
#     "Totoro20220531_.nwb",
#     "Totoro20220602_.nwb", #permissions
#     "Totoro20220603_.nwb",
#     "Totoro20220606_.nwb",
#     "Totoro20220607_.nwb",
#     "Totoro20220610_.nwb",
#     "Totoro20220613_.nwb",
#     "Totoro20220614_.nwb",
#     "Totoro20220615_.nwb",
#     "Totoro20220616_.nwb",
#     "Totoro20220617_.nwb",
#     # "Totoro20220621_.nwb", # Not inserted
#     #Olive
#     # "Olive20220624_.nwb",
#     "Olive20220627_.nwb",
#     "Olive20220628_.nwb",
#     "Olive20220629_.nwb",
#     "Olive20220630_.nwb",
#     "Olive20220701_.nwb",
#     "Olive20220702_.nwb",
#     "Olive20220705_.nwb",
#     "Olive20220706_.nwb",
#     "Olive20220707_.nwb",
#     "Olive20220708_.nwb",
#     "Olive20220711_.nwb",
#     #Winnie
#     "Winnie20220712_.nwb",
#     "Winnie20220713_.nwb",
#     "Winnie20220714_.nwb",
#     "Winnie20220715_.nwb",
#     "Winnie20220716_.nwb",
#     "Winnie20220717_.nwb",
#     "Winnie20220718_.nwb",
#     "Winnie20220719_.nwb",
#     "Winnie20220720_.nwb",
#     "Winnie20220722_.nwb",
#     #Wallie
#     "Wallie20220911_.nwb",
#     "Wallie20220912_.nwb",
#     "Wallie20220913_.nwb",
#     "Wallie20220914_.nwb",
#     "Wallie20220915_.nwb",
#     "Wallie20220916_.nwb",
#     "Wallie20220922_.nwb",
# ]

tetrode_animal_names = ["Wallie","Winnie","Olive","Totoro","Yoshi"]
tetrode_animal_names = ["Frodo"]
tetrode_animal_names = ["Bilbo","Winnie","Frodo"]
from spyglass.common import Session
tetrode_animals = []
[tetrode_animals.extend((Session() & {"subject_id":animal}).fetch("nwb_file_name")) for animal in tetrode_animal_names]
tetrode_animals = ["Winnie20220714_.nwb"]


#write permission?
analysis_path = "/stelmo/nwb/analysis/"
for nwb_file in tetrode_animals:
    if not os.path.isdir(analysis_path+nwb_file.split('_')[0]):
        print(f'No analysis yet for: {nwb_file}')
    elif not os.access(analysis_path+nwb_file.split('_')[0],os.W_OK):
        print(f'No write permissions for: {nwb_file} nwb_file')

# for nwb_file in probe_animals:
#     print(LFP() & {'nwb_file_name':nwb_file})

In [3]:
tetrode_animals = ['Frodo20230808_.nwb']

In [4]:
nwb_file = "Yoshi20220510_.nwb"
analysis_path+nwb_file.split('_')[0]

'/stelmo/nwb/analysis/Yoshi20220510'

# Run TetrodeAnimals LFP

In [7]:
nwb_file_name_list = tetrode_animals
filter_name = "LFP 0-400 Hz"
filter_sampling_rate = 30000
# band_filter_name = "Theta 5-11 Hz"
# band_sampling_rate = 100


analysis_path = "/stelmo/nwb/analysis/"
skipped_sessions = []
for nwb_file_name in nwb_file_name_list:
    # skip session if you don't have write privledges
    if os.path.isdir(analysis_path+nwb_file_name.split('_')[0]) and (not os.access(analysis_path+nwb_file_name.split('_')[0],os.W_OK)):
        skipped_sessions.append(nwb_file_name)
        continue
    print(f'Beginning: {nwb_file_name}')

    animal_name = nwb_file_name.split('202')[0]
    lfp_electrode_group_name = f'tetrode_sample_{animal_name}'
    #want to run on one electrode per tetrode
    electrode_ids = []
    lfp_eg_key = {
        "nwb_file_name": nwb_file_name,
        "lfp_electrode_group_name": lfp_electrode_group_name,
        }
    #loop through groups
    #identify the reference electrode. Make sure it's included
    ref_electrode = (Electrode & {"nwb_file_name": nwb_file_name}).fetch("original_reference_electrode")[0]
    e_groups = np.unique((Electrode & {"nwb_file_name": nwb_file_name}).fetch('electrode_group_name'))
    for e_group in e_groups:
        #loop through electrodes in group
        group_electrodes = (Electrode & {"nwb_file_name": nwb_file_name,'electrode_group_name':e_group}).fetch('electrode_id')
        for electrode in group_electrodes:
            if ref_electrode in group_electrodes:
                electrode_ids.append(ref_electrode)
                break
            #add the first good electrode to get LFP calculation
            if (Electrode & {"nwb_file_name": nwb_file_name,'electrode_group_name':e_group, 'electrode_id':electrode}).fetch1('bad_channel')=='False':
                electrode_ids.append(electrode)
                break
    #add an electrode group sampling each tetrode for this file
    # lfp_eg_key = {
    #     "nwb_file_name": nwb_file_name,
    #     "lfp_electrode_group_name": lfp_electrode_group_name,
    #     }
    LFPElectrodeGroup.create_lfp_electrode_group(
            nwb_file_name=nwb_file_name,
            group_name=lfp_electrode_group_name,
            electrode_list=electrode_ids,  
            )
    
    
    
    #Loop through all position intervals
    for interval_list_name in tqdm((IntervalList& {"nwb_file_name": nwb_file_name,}).fetch('interval_list_name')):        
        #make selection entry
        if ('pos' not in interval_list_name) or ('lfp' in interval_list_name) or (not (interval_list_name[:3]=='pos')): 
            continue
        print(interval_list_name)
        lfp_s_key = lfp_eg_key.copy()
        lfp_s_key["target_interval_list_name"] = interval_list_name
        lfp_s_key["filter_name"] = filter_name
        lfp_s_key["filter_sampling_rate"] = filter_sampling_rate
        if LFPV1()&lfp_s_key:
            continue #skip if already populated
        LFPSelection.insert1(lfp_s_key, skip_duplicates=True)
        LFPSelection()& {'nwb_file_name': nwb_file_name}
        #populate the lfp
        LFPV1().populate(lfp_s_key)

        # #Make band Filters
        # # set the reference to -1 to indicate no reference for all channels
        # ref_elect = [-1]
        # # we also need the uuid for the LFP object
        # lfp_id = (LFPOutput.LFP & lfp_s_key).fetch1("lfp_id")

        # LFPBandSelection().set_lfp_band_electrodes(
        #     nwb_file_name=nwb_file_name,
        #     lfp_id=lfp_id,
        #     electrode_list=electrode_ids,
        #     filter_name=band_filter_name,
        #     interval_list_name=interval_list_name,
        #     reference_electrode_list=ref_elect,
        #     lfp_band_sampling_rate=band_sampling_rate,
        # )
        # lfp_b_key = (
        #     LFPBandSelection & {"lfp_id": lfp_id, "filter_name": band_filter_name , 'target_interval_list_name':interval_list_name}
        # ).fetch1("KEY")
        # LFPBand().populate(LFPBandSelection() & {"nwb_file_name": nwb_file_name})
        # break



Beginning: Winnie20220714_.nwb


  0%|          | 0/31 [00:00<?, ?it/s][2023-10-26 11:22:27,940][WARNING]: Skipped checksum for file with hash: 72a61dc6-65af-acbb-56a4-80758ca9dbc0, and path: /stelmo/nwb/raw/Winnie20220714_.nwb


pos 0 valid times
LFP: found 44 intervals > 1.0 sec long.
Writing new NWB file Winnie20220714_ISXEFBV66J.nwb


Filtering data
Interval 0: loading data into memory
Interval 1: loading data into memory
Interval 2: loading data into memory
Interval 3: loading data into memory
Interval 4: loading data into memory
Interval 5: loading data into memory
Interval 6: loading data into memory
Interval 7: loading data into memory
Interval 8: loading data into memory
Interval 9: loading data into memory
Interval 10: loading data into memory
Interval 11: loading data into memory
Interval 12: loading data into memory
Interval 13: loading data into memory
Interval 14: loading data into memory
Interval 15: loading data into memory
Interval 16: loading data into memory
Interval 17: loading data into memory
Interval 18: loading data into memory
Interval 19: loading data into memory
Interval 20: loading data into memory
Interval 21: loading data into memory
Interval 22: loading data into memory
Interval 23: loading data into memory
Interval 24: loading data into memory
Interval 25: loading data into memory
Interva

 52%|█████▏    | 16/31 [06:32<06:08, 24.56s/it][2023-10-26 11:29:00,797][WARNING]: Skipped checksum for file with hash: 72a61dc6-65af-acbb-56a4-80758ca9dbc0, and path: /stelmo/nwb/raw/Winnie20220714_.nwb


...

 90%|█████████ | 28/31 [1:30:51<21:09, 423.13s/it][2023-10-26 12:53:19,640][WARNING]: Skipped checksum for file with hash: 72a61dc6-65af-acbb-56a4-80758ca9dbc0, and path: /stelmo/nwb/raw/Winnie20220714_.nwb


pos 8 valid times
LFP: found 63 intervals > 1.0 sec long.
Writing new NWB file Winnie20220714_YN6HXN35Y3.nwb
Filtering data
Interval 0: loading data into memory
Interval 1: loading data into memory
Interval 2: loading data into memory
Interval 3: loading data into memory
Interval 4: loading data into memory
Interval 5: loading data into memory
Interval 6: loading data into memory
Interval 7: loading data into memory
Interval 8: loading data into memory
Interval 9: loading data into memory
Interval 10: loading data into memory
Interval 11: loading data into memory
Interval 12: loading data into memory
Interval 13: loading data into memory
Interval 14: loading data into memory
Interval 15: loading data into memory
Interval 16: loading data into memory
Interval 17: loading data into memory
Interval 18: loading data into memory
Interval 19: loading data into memory
Interval 20: loading data into memory
Interval 21: loading data into memory
Interval 22: loading data into memory
Interval 23:

 94%|█████████▎| 29/31 [1:38:20<14:20, 430.40s/it][2023-10-26 13:00:47,928][WARNING]: Skipped checksum for file with hash: 72a61dc6-65af-acbb-56a4-80758ca9dbc0, and path: /stelmo/nwb/raw/Winnie20220714_.nwb


pos 9 valid times
LFP: found 53 intervals > 1.0 sec long.
Writing new NWB file Winnie20220714_8S8864K2DZ.nwb
Filtering data
Interval 0: loading data into memory
Interval 1: loading data into memory
Interval 2: loading data into memory
Interval 3: loading data into memory
Interval 4: loading data into memory
Interval 5: loading data into memory
Interval 6: loading data into memory
Interval 7: loading data into memory
Interval 8: loading data into memory
Interval 9: loading data into memory
Interval 10: loading data into memory
Interval 11: loading data into memory
Interval 12: loading data into memory
Interval 13: loading data into memory
Interval 14: loading data into memory
Interval 15: loading data into memory
Interval 16: loading data into memory
Interval 17: loading data into memory
Interval 18: loading data into memory
Interval 19: loading data into memory
Interval 20: loading data into memory
Interval 21: loading data into memory
Interval 22: loading data into memory
Interval 23:

100%|██████████| 31/31 [1:47:34<00:00, 208.20s/it]


In [4]:
%debug
# import os
# os.environ['PYTHONBREAKPOINT']

ERROR:root:No traceback has been produced, nothing to debug.


In [5]:
Session & {"nwb_file_name": nwb_file_name}
nwb_file_name
# interval_list_name

# (IntervalList & {"nwb_file_name": nwb_file_name, 'interval_list_name':interval_list_name}).fetch('valid_times')

# import spyglass.common as sgc
# (sgc.Raw& {"nwb_file_name": nwb_file_name}).fetch_nwb()

# LFPV1 & {"nwb_file_name":nwb_file_name}
# LFPSelection & {"nwb_file_name":nwb_file_name}

'Winnie_20220714_.nwb'

In [17]:
lfp_eg_key

{'nwb_file_name': 'Bilbo20230722_.nwb',
 'lfp_electrode_group_name': 'tetrode_sample_Bilbo'}

In [16]:
LFPElectrodeGroup.Electrode & {'lfp_electrode_group_name':'tetrode_sample_Bilbo'}

AttributeError: type object 'LFPElectrodeGroup' has no attribute 'Electrode'

In [10]:
LFPElectrodeGroup().fetch("lfp_electrode_group_name")#&{"nwb_file_name":"Winie20220710_.nwb",}

array(['all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'all_tets_arthur', 'all_tets_arthur', 'all_tets_arthur',
       'full_probe', 'full_probe', 'full_probe', 'full_probe',
       'full_probe', 'full_probe', 'full_probe', 'full_probe',
       'full_probe', 'full_probe', 'full_probe', 'full_probe',
       'full_probe', 'full_probe', 'full_probe', 'test_group',
       'full_probe', 'CH101_cortex', 'CH105_cortex', 'CH105_cortex',
       'CH105_cortex', 'CH105_cortex', 'CH109_cortex', 'CH112_cortex',
       'CH61_hippocampus', 'CH61_cortex', 'CH61_hippocampus',
       'CH61_cortex', 'CH61_hippocampus', 'CH61_cortex',
       'CH61_hippocampus', 'CH61_cortex',

In [31]:
lfp_s_key
# filter = (FirFilterParameters() & {"filter_name": lfp_s_key["filter_name"]} & {"filter_sampling_rate": filter_sampling_rate}).fetch(as_dict=True)
# filter[0]
# filter = (
#             FirFilterParameters()
#             & {"filter_name": lfp_s_key["filter_name"]}
#             & {"filter_sampling_rate": sampling_rate}
#         ).fetch(as_dict=True)[0]

from spyglass.common import Raw
(Raw & {'nwb_file_name':nwb_file_name}).fetch1(
            "sampling_rate", "interval_list_name"
        )
nwb_file_name

'Banner20220105_.nwb'

# Define Tetrode Animals

# Trial Junkyard

In [ ]:
LFPElectrodeGroup().LFPElectrode() & {"nwb_file_name": nwb_file_name}
IntervalList& {"nwb_file_name": nwb_file_name,}
# interval_list[:4]
LFP()#Selection()& {'nwb_file_name': nwb_file_name}
# lfp_s_key
LFPBandSelection()
interval_list_name

{"lfp_id": lfp_id, "filter_name": filter_name , 'target_interval_list_name':interval_list_name}
LFP()
(LFP() & lfp_s_key).fetch_nwb()[0]["lfp"]
(LFPBandSelection() & lfp_b_key)
LFPBand().populate(LFPBandSelection() & lfp_b_key)

In [26]:
lfp_interval_list = LFPOutput().get_lfp_object({"lfp_id": lfp_id}).fetch1("interval_list_name")
lfp_valid_times = (
            IntervalList()
            & {
                "nwb_file_name": nwb_file_name,
                "interval_list_name": lfp_interval_list,
            }
        ).fetch1("valid_times")

valid_times = (
            IntervalList()
            & {
                "nwb_file_name": nwb_file_name,
                "interval_list_name": interval_list_name,
            }
        ).fetch1("valid_times")

epoch_interval = (IntervalList()& {
                "nwb_file_name": nwb_file_name,
                "interval_list_name": interval_list_name,
            })
min_length = (LFPBandSelection & lfp_b_key).fetch1("min_interval_len")
from functools import reduce

interval_list_intersect(
            valid_times, lfp_valid_times, min_length=min_length
        )

lfp_object = LFPOutput.get_lfp_object({"lfp_id": lfp_id})
timestamps = np.asarray(lfp_object.timestamps)

AttributeError: 'LFP' object has no attribute 'timestamps'

In [14]:
LFPOutput().get_lfp_object({'nwb_file_name':nwb_file_name,"lfp_id": lfp_id})
(LFPBandSelection & lfp_b_key).fetch1("min_interval_len")

1.0

In [12]:
print(lfp_valid_times,'\n',valid_times,'\n',epoch_interval.fetch1('valid_times'))
# LFPOutput().get_lfp_object({"lfp_id": lfp_id,})

[[1.64573092e+09 1.64573187e+09]
 [1.64573187e+09 1.64573213e+09]] 
 [[1.64573092e+09 1.64573213e+09]] 
 [[1.64573092e+09 1.64573213e+09]]


In [29]:
a = [1.64573187e+09, 1.64573187e+09]
print(a[1]-a[0])
lfp_valid_times[1][0]-lfp_valid_times[0][1]
lfp_object = (
            LFP() & {"nwb_file_name": nwb_file_name}
        ).fetch_nwb()[0]["lfp"]

[2023-06-30 16:26:14,399][WARNING]: Skipped checksum for file with hash: 8da8b253-bd8f-ca99-e332-91a506d6bac6, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_VCWQJUGZXX.nwb
[2023-06-30 16:26:14,404][WARNING]: Skipped checksum for file with hash: 7bcb1c34-7c3a-54aa-4456-791f9c23b739, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_ILSBCE5BER.nwb
[2023-06-30 16:26:14,408][WARNING]: Skipped checksum for file with hash: 05192d1d-1fe8-dff3-efea-1b2b3ecaaa3b, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_L482ADMDTN.nwb
[2023-06-30 16:26:14,412][WARNING]: Skipped checksum for file with hash: a3c1af13-b53f-caf9-4ce1-3ee16276045d, and path: /stelmo/nwb/analysis/Banner20220224/Banner20220224_XFMG8I2JA3.nwb


0.0


In [30]:
LFP() & {"nwb_file_name": nwb_file_name}

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
Banner20220224_.nwb,full_probe,pos 0 valid times,LFP 0-400 Hz,30000,Banner20220224_VCWQJUGZXX.nwb,full_probe_pos 0 valid times_valid times,0a0a6802-0724-4d4b-8985-d6732af52179,1000.0
Banner20220224_.nwb,full_probe,pos 1 valid times,LFP 0-400 Hz,30000,Banner20220224_ILSBCE5BER.nwb,full_probe_pos 1 valid times_valid times,b91f0624-43d8-4a3f-aeb5-c167fd98d3c3,1000.0
Banner20220224_.nwb,test_group,pos 3 valid times,LFP 0-400 Hz,30000,Banner20220224_L482ADMDTN.nwb,test_group_pos 3 valid times_valid times,9769ba64-6ecd-43bf-89cd-fccf32c9977b,1000.0
Banner20220224_.nwb,test_group,test interval,LFP 0-400 Hz,30000,Banner20220224_XFMG8I2JA3.nwb,test_group_test interval_valid times,ddfa2e79-b188-40bb-92cd-3698d35a204b,1000.0


In [23]:
from spyglass.common.common_interval import *